# Голумин Никита РТ5-61Б РК2(Вариант 6)

## Задание
Для заданного набора данных постройте модели классификации или регрессии. Для построения моделей используйте методы Дерево решений и Градиентный бустинг. Оцените качество моделей на основе подходящих метрик качества (не менее двух метрик). Какие метрики качества Вы использовали и почему? Какие выводы Вы можете сделать о качестве построенных моделей? Для построения моделей необходимо выполнить требуемую предобработку данных: заполнение пропусков, кодирование категориальных признаков, и т.д.

# Прогнозирование шансов поступления в аспирантуру

В данном проекте мы будем использовать набор данных о поступлении в аспирантуру в Индии, доступный по ссылке: [Kaggle Graduate Admissions Dataset](https://www.kaggle.com/datasets/mohansacharya/graduate-admissions?resource=download)

Этот проект может помочь выпускникам оценить свои шансы на поступление и выявить ключевые факторы, влияющие на этот процесс.

## Описание датасета

Датасет включает один файл `Admission_Predict.csv` с такими колонками:

- **GRE Scores (из 340)**: результат теста GRE
- **TOEFL Scores (из 120)**: результат теста TOEFL
- **University Rating (из 5)**: рейтинг университета
- **Statement of Purpose (из 5)**: оценка мотивационного письма
- **Undergraduate GPA (из 10)**: средний балл за бакалавриат
- **Research Experience (0 или 1)**: наличие исследовательского опыта
- **Chance of Admit (от 0 до 1)**: вероятность поступления

## Цель задачи

Цель данной задачи регрессии — предсказать вероятность поступления (`Chance of Admit`) на основе предоставленных признаков.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style="ticks")

In [2]:
data = pd.read_csv('Admission_Predict.csv')

In [3]:
# Удалим дубликаты записей, если они присутствуют
data = data.drop_duplicates()

In [4]:
# Первые 5 строк датасета
data.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [5]:
# Размер  датасета - 400 строк, 9 колонок
data.shape

(400, 9)

In [6]:
# Список колонок с типами данных 
data.dtypes

Serial No.             int64
GRE Score              int64
TOEFL Score            int64
University Rating      int64
SOP                  float64
LOR                  float64
CGPA                 float64
Research               int64
Chance of Admit      float64
dtype: object

In [7]:
# Проверим наличие пустых значений
data.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

### Удалим столбец "Serial No.", который не нагружен никакой полезной информацией для анализа данных

In [8]:
del data["Serial No."]
data.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,337,118,4,4.5,4.5,9.65,1,0.92
1,324,107,4,4.0,4.5,8.87,1,0.76
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65


### Выбор метрик для последующей оценки качества моделей:
- Mean absolute error - средняя абсолютная ошибка
- Mean squared error - средняя квадратичная ошибка
- R2 - коэффициент детерминации

**MAE** и **MSE** взяты как как важна направленность ошибки, **R2** так как важно, насколько хорошо модель объясняет вариацию в данных

### Выбор моделей:
- Решающее дерево
- Градиентный бустинг

### Решающее дерево

In [9]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [10]:
# Признаки без целевой переменной
x = data.drop('Chance of Admit ', axis=1)
x.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337,118,4,4.5,4.5,9.65,1
1,324,107,4,4.0,4.5,8.87,1
2,316,104,3,3.0,3.5,8.00,1
3,322,110,3,3.5,2.5,8.67,1
4,314,103,2,2.0,3.0,8.21,0


In [11]:
# Целевая переменная
y = data['Chance of Admit ']
y.head()

0    0.92
1    0.76
2    0.72
3    0.80
4    0.65
Name: Chance of Admit , dtype: float64

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
#20% данных будут отложены для тестирования, а 80% будут использованы для обучения модели

In [13]:
# Размер обучающей выборки
x_train.shape, y_train.shape

((320, 7), (320,))

In [14]:
# Размер тестовой выборки
x_test.shape, y_test.shape

((80, 7), (80,))

In [15]:
model = DecisionTreeRegressor(random_state=3)
model.fit(x_train, y_train)

DecisionTreeRegressor(random_state=3)

In [16]:
y_pred = model.predict(x_test)

In [17]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [18]:
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R² Score: {r2}")


Mean Absolute Error (MAE): 0.06487499999999999
Mean Squared Error (MSE): 0.007751249999999999
R² Score: 0.6648461378395251


### Градиентный бустинг

In [19]:
from sklearn.ensemble import GradientBoostingRegressor

In [20]:
gradient_boosting_regressor= GradientBoostingRegressor()
gradient_boosting_regressor.fit(x_train, y_train)

GradientBoostingRegressor()

In [21]:
y_grad_pred = gradient_boosting_regressor.predict(x_test)

In [22]:
mae = mean_absolute_error(y_test, y_grad_pred)
mse = mean_squared_error(y_test, y_grad_pred)
r2 = r2_score(y_test, y_grad_pred)

In [23]:
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R² Score: {r2}")

Mean Absolute Error (MAE): 0.04572411166507685
Mean Squared Error (MSE): 0.0049014371065016025
R² Score: 0.7880683017086695


### Общий вывод

Модель градиентного бустинга демонстрирует лучшие результаты по всем метрикам по сравнению с моделью решающего дерева. Это указывает на то, что градиентный бустинг является более точной и надежной моделью для предсказания данной задачи.

- **Градиентный бустинг** имеет меньшие ошибки (MAE и MSE) и более высокий коэффициент детерминации (R²), что свидетельствует о более высоком качестве предсказаний.
- **Решающее дерево** показывает худшие результаты по сравнению с градиентным бустингом, что может быть связано с его склонностью к переобучению и меньшей гибкостью в моделировании сложных зависимостей.

Следовательно, модель градиентного бустинга предпочтительнее использовать для данной задачи, так как она обеспечивает более высокую точность и объясняющую способность.
